<a href="https://colab.research.google.com/github/ClesteA/MedSimulator/blob/main/vllmcasegen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# --- 1. KURULUM (vLLM Engine) ---
# vLLM, şu an dünyadaki en hızlı LLM servis kütüphanesidir.
!pip install -q vllm

import json
import random
import re
from vllm import LLM, SamplingParams
from huggingface_hub import login
from google.colab import files

# Token Girişi
print("Hugging Face Token Giriniz:")
login()

# --- 2. AYARLAR ---
MODEL_ID = "google/medgemma-27b-text-it"

# A100 80GB olduğu için modeli sıkıştırmadan (bfloat16) yükleyebiliriz.
# Bu en yüksek kaliteyi ve hızı verir.
print(f"\n🚀 vLLM Motoru Başlatılıyor... (Model: {MODEL_ID})")

# GPU Kullanım Ayarları
llm = LLM(
    model=MODEL_ID,
    dtype="bfloat16",       # A100'ün ana dili, en hızlısı
    tensor_parallel_size=1, # Tek GPU kullanıyoruz
    gpu_memory_utilization=0.95, # VRAM'in %95'ini kullan (Yaklaşık 76GB)
    max_model_len=4096,     # Context penceresi
    enforce_eager=True      # CUDA Graph hatasını önlemek için güvenli mod
)

# Üretim Parametreleri
sampling_params = SamplingParams(
    temperature=0.55,
    top_p=0.95,
    max_tokens=2048, # Her vaka için maksimum uzunluk
    stop=["<end_of_turn>"] # Modelin durması gereken yer
)

# --- 3. PROMPT HAZIRLAMA ---
def prompt_olustur(tani):
    return f"""<start_of_turn>user
    Sen klinik simülasyon senaryoları ve vakaları hazırlayan kıdemli bir tıp profesörüsün.

            GÖREV: '{tani}' tanısı için tıp öğrencileri ve doktorları eğitecek zorlukta ve gerçekçilikte bir vaka oluştur. Tutarlılığa maksimum önem ver.

            KRİTİK KURALLAR (Bunlara kesinlikle uy):
            1. **Epidemiyoloji:** Hastanın yaşı ve cinsiyeti, '{tani}' hastalığının gerçek hayattaki insidansına uygun olmalı.
            2. **Dil Ayrımı:** - "sikayet" alanı: Hastanın kendi ağzından, basit, korkmuş bir halk diliyle yazılmalı. (Örn: "Hocam göğsüme bir fil oturdu sanki")
               - Diğer tüm alanlar: Tamamen profesyonel, akademik TIBBİ TERMİNOLOJİ ile yazılmalı.
            3. **Tutarlılık:** Vital bulgular hastalığın şiddetiyle uyumlu olmalı. (Örn: Şok tablosundaysa TA düşük, Nabız yüksek olmalı).
            4. **Laboratuvar:** Sadece patolojik olanları değil, kafa karıştırmak için normal çıkan rutin tetkikleri de listeye ekle. Birimleri (mg/dL vb.) mutlaka yaz.
            5. **Çıktı:** SADECE JSON formatında yanıt ver. Yorum yapma.

            JSON ŞEMASI:
            {{
                "id": "vaka_ID",
                "gizli_tani": "{tani}",
                "hasta_kimlik": {{
                    "ad_soyad": "Rastgele Türk İsmi",
                    "yas": "integer",
                    "cinsiyet": "Erkek/Kadın",
                    "meslek": "Rastgele meslek. Bazen hastalıkla ilişkili uygun meslek de seçilebilir.",
                    "sikayet": "Hastanın ilk cümlesi (Halk ağzı)"
                }},
                "anamnez": {{
                    "sikayet_detaylari": "Tıbbi dille detaylı anamnez (Süre, karakter, yayılım, eşlik eden semptomlar)",
                    "kronik_hastaliklar": "Varsa ek hastalıklar",
                    "kullandigi_ilaclar": "İlaçlar ve dozları",
                    "tibbi_ozgecmis": "Operasyonlar, alerjiler"
                }},
                "bulgular": {{
                    "fizik_muayene": "Sistemik muayene (Genel durum, Vital Bulgular: TA, Nabız, Ateş, SpO2, Solunum sayısı mutlaka olsun)",
                    "laboratuvar": "Kan/İdrar sonuçları (Birimleriyle)",
                    "goruntuleme": "Radyoloji veya EKG rapor sonucu"
                }}
            }}<end_of_turn>
<start_of_turn>model
            """

# --- 4. TOPLU ÜRETİM (BATCH PROCESSING) ---
tanilar = [
    "Akut Pankreatit", "Diyabetik Ketoasidoz (DKA)", "Toplum Kökenli Pnömoni",
    "Akut Piyelonefrit", "Demir Eksikliği Anemisi", "Pulmoner Emboli", "Hipertansif Acil",
    "Kronik Obstrüktif Akciğer Hastalığı (KOAH) Alevlenmesi", "Tansiyonel Pnömotoraks",
    "Subaraknoid Kanama", "Hipokalemi", "Akut Apandisit", "Akut Böbrek Hasarı",
    "Kafa Travması", "Hiponatremi", "Dismenore", "Serebrovasküler Olay (SVO)",
    "Kemik Kırığı", "Akut Glomerülonefrit", "Tirotoksikoz", "Atriyal Fibrilasyon",
    "Akut Astım Atağı", "Perikardit", "Akut Ürtiker", "Supraventriküler Taşikardi (SVT)",
    "Bradikardi", "Ventriküler Taşikardi", "Ventriküler Fibrilasyon", "Hiperkalemi",
    "Akut Pulmoner Ödem", "Akut Karaciğer Yetmezliği", "Akut Menenjit", "Akut Kolanjit",
    "Akut Divertikülit", "Akut Hipotansiyon", "Akut Nazofarenjit (ÜSYE)",
    "Akut Tonsillofarenjit", "Akut Sinüzit", "İnfluenza", "Alerjik Rinit",
    "Miyokard Enfarktüsü (USAP-NSTEMI-STEMI)", "Derin Ven Trombozu", "Gastroözofageal Reflü",
    "Akut Gastrit", "Akut Gastroenterit", "Akut Kolesistit", "Hemoroidal Hastalık",
    "Dislipidemi", "Gut Atağı", "Migren", "Gerilim Tipi Baş Ağrısı", "BPPV",
    "Vestibüler Nörit", "Epilepsi Nöbeti", "Karpal Tünel Sendromu", "Bell Paralizisi",
    "Akut Sistit", "Renal Kolik", "Mekanik Bel Ağrısı", "Lomber Disk Hernisi",
    "Osteoartrit", "Fibromiyalji", "Ayak Bileği Burkulması", "Akut Otitis Media",
    "Akut Otitis Eksterna", "Epistaksis", "Konjonktivit", "Panik Atak", "Anafilaksi",
    "Yanıklar", "Aort Diseksiyonu"
]

# Prompt Listesini Hazırla (Her tanıdan 3 adet)
all_prompts = []
tani_meta = [] # Hangi promptun hangi tanıya ait olduğunu tutar

ADET_BASI = 10

print(f"\n⚡ {len(tanilar) * ADET_BASI} adet vaka için promptlar hazırlanıyor...")
for tani in tanilar:
    for _ in range(ADET_BASI):
        all_prompts.append(prompt_olustur(tani))
        tani_meta.append(tani)

# --- MOTORU ATEŞLE ---
print(f"🔥 BATCH GENERATION BAŞLIYOR... (Arkanıza yaslanın, vLLM A100'ü ağlatacak)")
outputs = llm.generate(all_prompts, sampling_params)

# --- 5. SONUÇLARI İŞLE ---
vaka_veritabani = []
global_sayac = 1

def extract_json_clean(text):
    try:
        text = re.sub(r'```json\s*', '', text)
        text = re.sub(r'```', '', text)
        start = text.find('{')
        end = text.rfind('}') + 1
        if start != -1 and end != -1:
            return json.loads(text[start:end])
    except:
        return None

print("\n✅ Üretim tamamlandı. JSON formatına dönüştürülüyor...")

for i, output in enumerate(outputs):
    generated_text = output.outputs[0].text
    tani_ismi = tani_meta[i]

    data = extract_json_clean(generated_text)

    if data:
        data["id"] = f"vaka_{global_sayac:03d}"
        data["gizli_tani"] = tani_ismi
        vaka_veritabani.append(data)
        global_sayac += 1
    else:
        print(f"⚠️ Format Hatası (Index {i} - {tani_ismi})")

# Karıştır
random.shuffle(vaka_veritabani)

# Kaydet
dosya_adi = "medsim_vllm_ultra_fast.json"
with open(dosya_adi, "w", encoding="utf-8") as f:
    json.dump(vaka_veritabani, f, ensure_ascii=False, indent=4)

print(f"\n🎉 İŞLEM BİTTİ! Toplam {len(vaka_veritabani)} vaka üretildi.")
print(f"Dosya: {dosya_adi}")
files.download(dosya_adi)

Hugging Face Token Giriniz:



🚀 vLLM Motoru Başlatılıyor... (Model: google/medgemma-27b-text-it)
INFO 12-08 12:51:52 [utils.py:253] non-default args: {'dtype': 'bfloat16', 'seed': None, 'max_model_len': 4096, 'gpu_memory_utilization': 0.95, 'disable_log_stats': True, 'enforce_eager': True, 'model': 'google/medgemma-27b-text-it'}


config.json:   0%|          | 0.00/931 [00:00<?, ?B/s]

WARNING 12-08 12:51:52 [arg_utils.py:1175] `seed=None` is equivalent to `seed=0` in V1 Engine. You will no longer be allowed to pass `None` in v0.13.
INFO 12-08 12:52:12 [model.py:637] Resolved architecture: Gemma3ForCausalLM
INFO 12-08 12:52:12 [model.py:1750] Using max model len 4096
INFO 12-08 12:52:15 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 12-08 12:52:15 [vllm.py:601] Enforce eager set, overriding optimization level to -O0
INFO 12-08 12:52:15 [vllm.py:707] Cudagraph is disabled under eager mode


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

...

Adding requests:   0%|          | 0/710 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/710 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


✅ Üretim tamamlandı. JSON formatına dönüştürülüyor...
⚠️ Format Hatası (Index 0 - Akut Pankreatit)
⚠️ Format Hatası (Index 1 - Akut Pankreatit)
⚠️ Format Hatası (Index 2 - Akut Pankreatit)
⚠️ Format Hatası (Index 3 - Akut Pankreatit)
⚠️ Format Hatası (Index 8 - Akut Pankreatit)
⚠️ Format Hatası (Index 9 - Akut Pankreatit)
⚠️ Format Hatası (Index 10 - Diyabetik Ketoasidoz (DKA))
⚠️ Format Hatası (Index 13 - Diyabetik Ketoasidoz (DKA))
⚠️ Format Hatası (Index 14 - Diyabetik Ketoasidoz (DKA))
⚠️ Format Hatası (Index 20 - Toplum Kökenli Pnömoni)
⚠️ Format Hatası (Index 21 - Toplum Kökenli Pnömoni)
⚠️ Format Hatası (Index 23 - Toplum Kökenli Pnömoni)
⚠️ Format Hatası (Index 27 - Toplum Kökenli Pnömoni)
⚠️ Format Hatası (Index 30 - Akut Piyelonefrit)
⚠️ Format Hatası (Index 31 - Akut Piyelonefrit)
⚠️ Format Hatası (Index 32 - Akut Piyelonefrit)
⚠️ Format Hatası (Index 33 - Akut Piyelonefrit)
⚠️ Format Hatası (Index 34 - Akut Piyelonefrit)
⚠️ Format Hatası (Index 35 - Akut Piyelonefrit)
⚠️ F

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>